# Scrapper para obtener cédula profesional

## Instalación de librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from funciones import *

## Buscar por registro

In [2]:
url = 'https://cedulaprofesionalsep.online/#Consulta_de_Cedula_Profesional'
driver = webdriver.Chrome()
iniciar_scraper(driver, url, wait_condition=True, class_name='tp-gateway-button')
resultado = cédula_profesional(driver, "Luis", "Cruz", "Jaramillo")
print(resultado)

Cantidad de registros: 2
Cantidad de registros: 2
None


## Buscar por csv

In [ ]:
init = 0
final = 2000
excel_file = 'Z:/Data/HSBC/Assignments/ASG_TDC_IA_JAN_JUNE.xlsx'
result_df = process_excel_file(init, final, excel_file)
print(result_df.head())

In [ ]:
init=0
final=2000

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: X.replace('/', ' ').replace('*', '').rstrip())
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered).iloc[init:final]



df_input['NombreSplit'] = df_input.Nombre.apply(lambda X: split_name(X))
df_input['Nombre'] = df_input.NombreSplit.apply(lambda X: X[0])
df_input['Paterno'] = df_input.NombreSplit.apply(lambda X: X[1])
df_input['Materno'] = df_input.NombreSplit.apply(lambda X: X[2])
# Cambiar nombre
#df.rename(columns={'Nombre':'Nombre'}, inplace=True)
df_input.head()

,Nombre,NombreSplit,Paterno,Materno
0,raul emanuelle,"[raul emanuelle, segura, miranda]",segura,miranda
1,carlos,"[carlos, molinar, varela]",molinar,varela
2,rene de jesus,"[rene de jesus, mena, chavez]",mena,chavez
3,ana luisa,"[ana luisa, aparicio, ruiz]",aparicio,ruiz
4,maria de la luz eugenia,"[maria de la luz eugenia, rivero, aguilar]",rivero,aguilar


### Scrapper

In [ ]:
data=[]
for index, row in df.iterrows():
    nombre=row['Nombre']
    paterno=row['Paterno']
    materno=row['Materno']
    # Ingresar nombres
    celda_nombre = driver.find_element(By.XPATH, '//*[@id="input-nombre"]')
    celda_nombre.send_keys(str(nombre))

    #Ingresar Apellido Paterno
    celda_paterno = driver.find_element(By.XPATH, '//*[@id="input-apaterno"]')
    celda_paterno.send_keys(str(paterno))

    # Ingresar Apellido Materno
    celda_materno = driver.find_element(By.XPATH, '//*[@id="input-amaterno"]')
    celda_materno.send_keys(str(materno))

    # Click en botón buscar
    buscar = driver.find_element(By.XPATH, '//*[@id="container-form-1"]/form/div[4]/div/input')
    buscar.click()
    time.sleep(2)

    max_wait_time = 5  # Tiempo máximo de espera en segundos
    start_time = time.time()  # Momento de inicio de espera

    while time.time() - start_time < max_wait_time:
        try:
            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            break  # Salir del bucle si se encuentra la ventana emergente
        except:

            time.sleep(1)  # Esperar 1 segundo antes de intentar nuevamente
            
            

    #tabla = driver.find_elements(By.XPATH, '//*[@id="result_table_row"]/tr[1]/td[3]/input')
        
        # Encontrar botones ver-mas
    ver_botones = driver.find_elements(By.CLASS_NAME, 'ver-mas')
        # Iterar sobre los botones
    
    
    for ver in ver_botones:
        cantidad_registros = len(ver_botones)
        print("Cantidad de registros:", cantidad_registros)

        #time.sleep()
        ver.click()
        
        # Encontrar los elementos de entrada de texto por su id
        input_cedula = driver.find_element(By.ID, "input-cedula-result")
        input_tipo = driver.find_element(By.ID, "input-tipo-result")
        input_sexo = driver.find_element(By.ID, "input-sexo-result")
        input_nombre = driver.find_element(By.ID, "input-nombre-result")
        input_paterno = driver.find_element(By.ID, "input-paterno-result")
        input_materno = driver.find_element(By.ID, "input-materno-result")
        input_escuela = driver.find_element(By.ID, "input-escuela-result")
        input_titulo = driver.find_element(By.ID, "input-titulo-result")

        # Obtener los valores de los campos
        valor_cedula = input_cedula.get_attribute("value")
        valor_tipo = input_tipo.get_attribute("value")
        valor_sexo = input_sexo.get_attribute("value")
        valor_nombre = input_nombre.get_attribute("value")
        valor_paterno = input_paterno.get_attribute("value")
        valor_materno = input_materno.get_attribute("value")
        valor_escuela = input_escuela.get_attribute("value")
        valor_titulo = input_titulo.get_attribute("value")

        # Agregar los valores a la lista de diccionarios
        data.append({
        "Cédula": valor_cedula,
        "Tipo de Cédula": valor_tipo,
        "Sexo": valor_sexo,
        "Nombre": valor_nombre,
        "Paterno": valor_paterno,
        "Materno": valor_materno,
        "Escuela": valor_escuela,
        "Título": valor_titulo
        })
        # Encontrar los elementos de entrada de texto por su id
        cerrar = driver.find_element(By.XPATH, '//*[@id="cerrar_resut_personal"]')
        cerrar.click()
    driver.refresh()

        
df_data = pd.DataFrame(data)
    

Cantidad de registros: 2
Cantidad de registros: 2
Cantidad de registros: 2
Cantidad de registros: 2
